In [5]:
%%writefile requirements.txt
pandas
matplotlib
seaborn
python-dotenv
langchain-community
langchain-openai
langchain-chroma
transformers
gradio
notebook
ipywidgets

Overwriting requirements.txt


In [6]:
!pip install -r ./requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.7 MB/s eta 0:00

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [9]:
from google.colab import userdata
import os

openai_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_key

In [10]:
# Let's load the cleaned books csv file from github
from pathlib import Path

import requests
import os

def download_data(source: str) -> Path:
    """Downloads a zipped dataset from source and unzips to a data file.

    Args:
        source (str): A link to a zipped file containing data.
        remove_source (bool): Whether to remove the source after downloading and extracting.

    Returns:
        pathlib.Path to downloaded data.

    Example usage:
        download_data(source="https://github.com/vvpokhilko/llm-semantic-book-recommender/raw/main/data/books_cleaned.csv",
                      destination="pizza_steak_sushi")
    """

    # Setup path to data folder
    data_path = Path("data/")

    # If the image folder doesn't exist, download it and prepare it...
    if data_path.is_dir() and os.listdir(data_path):
        print(f"[INFO] {data_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {data_path} directory, creating one...")
        data_path.mkdir(parents=True, exist_ok=True)

        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
          request = requests.get(source)
          print(f"[INFO] Downloading {target_file} from {source}...")
          f.write(request.content)

    return data_path

data_path = download_data(source="https://github.com/vvpokhilko/llm-semantic-book-recommender/raw/main/data/books_cleaned.csv")
data_path

[INFO] Did not find data directory, creating one...
[INFO] Downloading books_cleaned.csv from https://github.com/vvpokhilko/llm-semantic-book-recommender/raw/main/data/books_cleaned.csv...


PosixPath('data')

In [11]:
import pandas as pd

books = pd.read_csv("data/books_cleaned.csv")

In [12]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [13]:
books["tagged_description"]

,tagged_description
0,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982 A new 'Christie for Christmas' -...
2,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897 Lewis' work on the nature of lov...
4,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...
5192,9788172235222 On A Train Journey Home To North...
5193,9788173031014 This book tells the tale of a ma...
5194,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535 This collection of the timeless ...


In [14]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [ ]:
# Let's create data chunks
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [19]:
# Display a chunk of data for visualization
documents[5]

Document(metadata={'source': 'tagged_description.txt'}, page_content="9780006380832 Until Vasco da Gama discovered the sea-route to the East in 1497-9 almost nothing was known in the West of the exotic cultures and wealth of the Indian Ocean and its peoples. It is this civilization and its destruction at the hands of the West that Richard Hall recreates in this book. Hall's history of the exploration and exploitation by Chinese and Arab travellers, and by the Portuguese, Dutch and British alike is one of brutality, betrayal and colonial ambition.")

In [20]:
# Create a vector database
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [27]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs, len(docs)

([Document(id='b3662f2a-cfd1-48f4-928b-3421777db5e7', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
  Document(id='f0113c4a-0177-45ea-a640-01a7da9a334b', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
  Document(id='34b46be2-82ae-4ed0-900b-c8f4129366e2', metadata={'source'

In [26]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip('"'))]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [30]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = top_k)

    books_list = []

    for i in range(len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [31]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
3214,9780689861130,0689861133,"Moo, Baa, la la La!",Sandra Boynton,Animal sounds,http://books.google.com/books/content?id=Gz40A...,Children will love joining in and imitating th...,2004.0,4.20,14.0,28261.0,"Moo, Baa, la la La!",9780689861130 Children will love joining in an...
3581,9780763620875,0763620874,Judy Moody Saves the World!,Megan McDonald,Juvenile Fiction,http://books.google.com/books/content?id=xDIRB...,When Judy Moody gets serious about protecting ...,2004.0,4.03,160.0,5883.0,Judy Moody Saves the World!,9780763620875 When Judy Moody gets serious abo...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373 Introducing your baby to birds, ..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."
3751,9780786808717,0786808713,Baby Einstein: What Does Violet See? Raindrops...,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=95IIA...,A very special puddle sets Violet the mouse of...,2002.0,3.25,18.0,16.0,Baby Einstein: What Does Violet See? Raindrops...,9780786808717 A very special puddle sets Viole...
3760,9780786812912,0786812915,The Big Box,Toni Morrison;Slade Morrison,Juvenile Fiction,http://books.google.com/books/content?id=LyYKA...,"In her first illustrated book for children, th...",2002.0,3.95,48.0,375.0,The Big Box,9780786812912 In her first illustrated book fo...
3797,9780789458209,0789458209,Tree,David Burnie,Juvenile Nonfiction,http://books.google.com/books/content?id=Qwsqj...,Photographs and text explore the anatomy and l...,2000.0,4.07,64.0,5.0,Tree,9780789458209 Photographs and text explore the...


In [32]:
retrieve_semantic_recommendations("A book about pirate adventures")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
1276,9780312864743,0312864744,Ports of Call,Jack Vance,Fiction,http://books.google.com/books/content?id=_Zu4S...,A romantic tale follows a space swashbuckler a...,1999.0,3.53,300.0,395.0,Ports of Call,9780312864743 A romantic tale follows a space ...
2066,9780425205594,0425205592,Dark Watch,Clive Cussler;Jack B. Du Brul,Fiction,http://books.google.com/books/content?id=EKDrH...,When Japanese shipping magnates whose fortunes...,2005.0,4.09,357.0,5749.0,Dark Watch,9780425205594 When Japanese shipping magnates ...
2810,9780571164660,0571164668,The White Castle,Orhan Pamuk,Civil law,http://books.google.com/books/content?id=Y55qQ...,Winner of the 1990 Independent Award for forei...,1990.0,3.45,161.0,6669.0,The White Castle,9780571164660 Winner of the 1990 Independent A...
4437,9781402713187,1402713185,Treasure Island,Chris Tait;Robert Louis Stevenson;Lucy Corvino,Juvenile Fiction,http://books.google.com/books/content?id=NTGDz...,While going through the possessions of a decea...,2005.0,4.12,160.0,665.0,Treasure Island,9781402713187 While going through the possessi...
4440,9781402714573,1402714572,Treasure Island,Robert Louis Stevenson,Juvenile Fiction,http://books.google.com/books/content?id=UUl7Z...,While going through the possessions of a decea...,2004.0,3.83,213.0,393.0,Treasure Island,9781402714573 While going through the possessi...
4446,9781402732980,1402732988,Off to Sea,Robert Louis Stevenson;Catherine Nichols,Juvenile Fiction,http://books.google.com/books/content?id=vSY8f...,"A brief, simplified retelling of the episode i...",2006.0,4.17,32.0,6.0,Off to Sea,"9781402732980 A brief, simplified retelling of..."
4545,9781423100188,1423100182,Pirates of the Caribbean: The Coming Storm - J...,Rob Kidd,Juvenile Fiction,http://books.google.com/books/content?id=UkvjA...,Teenage stowaway Jack Sparrow and his band of ...,2006.0,3.83,135.0,1364.0,Pirates of the Caribbean: The Coming Storm - J...,9781423100188 Teenage stowaway Jack Sparrow an...
4546,9781423100195,1423100190,Pirates of the Caribbean: The Siren Song - Jac...,Rob Kidd,Juvenile Fiction,http://books.google.com/books/content?id=_7OzA...,Still on a mission to find the legendary Sword...,2006.0,3.86,123.0,832.0,Pirates of the Caribbean: The Siren Song - Jac...,9781423100195 Still on a mission to find the l...
4547,9781423100201,1423100204,Pirates of the Caribbean: Jack Sparrow #3 - th...,Rob Kidd,Juvenile Fiction,http://books.google.com/books/content?id=vOQgu...,Jack and company are hot on the trail of the n...,2006.0,3.89,119.0,713.0,Pirates of the Caribbean: Jack Sparrow #3 - th...,9781423100201 Jack and company are hot on the ...
4550,9781423101680,1423101685,Pirates of the Caribbean: Age of Bronze - Jack...,Rob Kidd,Juvenile Fiction,http://books.google.com/books/content?id=iHKIN...,The beginning of a new story arc! Jack and com...,2006.0,4.05,144.0,532.0,Pirates of the Caribbean: Age of Bronze - Jack...,9781423101680 The beginning of a new story arc...
